# Simulate a qualitative interview
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for simulating an interview between a researcher and a subject, with instructions for modifying the interviewer, interview subject or topic.

Tthe `Conversation` [module](https://github.com/expectedparrot/edsl/tree/main/edsl/conversation) can also be used to automate methods used below to simulate a conversation with multiple agents. See examples:

* [Buying a car](https://github.com/expectedparrot/edsl/blob/main/edsl/conversation/car_buying.py)
* [Negotiating a mug](https://github.com/expectedparrot/edsl/blob/main/edsl/conversation/mug_negotiation.py)

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Import the tools
Here we import the tools that we will use to conduct the interview. The interview is designed as a series of free text questions administered to agents representing the interviewer and subject. We use "scenarios" to parameterize the survey questions with prior content of the survey as the questions progress. Learn more about [EDSL question types](https://docs.expectedparrot.com/en/latest/questions.html) and other survey components.

In [1]:
from edsl import QuestionFreeText, Scenario, Survey, Model, Agent

import textwrap
from rich import print

EDSL works with many popular language models. Learn more about [selecting models](https://docs.expectedparrot.com/en/latest/language_models.html) to use with your surveys. To see a complete current list of available models, uncomment and run the following code:

In [2]:
# Model.available()

Here we select a model to use for the interview:

In [3]:
model = Model("gemini-2.5-flash")

## Create interview components
Edit the inputs in the following code block to change the instructions for the agent interviewer, the interview topic and/or the interview subject:

In [4]:
# A name for the interview subject
interview_subject_name = "Chicken"

# Traits of the interview subject
interview_subject_traits = {
    "persona": "You are a brave, independent-minded chicken.",
    "status": "wild",
    "home": "A free range farm some miles away.",
    "number_of_chicks": 12,
}

# Description of the interview topic
interview_topic = "Reasons to cross the road"

# Total number of questions to ask in the interview
total_questions = 5

# Description of the interviewer agent
interviewer_background = textwrap.dedent(
    f"""\
You are an expert qualitative researcher.  
You are conducting interviews to learn people's views on the following topic: {interview_topic}.
"""
)

# Instructions for the interviewer agent
interviewer_instructions = textwrap.dedent(
    """\
You know to ask questions that are appropriate to the age and experience of an interview subject.
You know to not ask questions that an interview subject would not be able to answer, 
e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
You ask excellent follow-up questions.
"""
)

## Interview methods
Here we create methods for constructing agents representing a researcher and subject, and conducting an interview between them in the form of a series of EDSL survey questions. Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [running surveys](https://docs.expectedparrot.com/en/latest/surveys.html).

In [5]:
def construct_subject(name, traits={}):
    return Agent(name=name, traits=traits)


def construct_researcher(interview_topic):
    return Agent(
        traits={"background": interviewer_background},
        instruction=interviewer_instructions,
    )


def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario(
        {"subject": str(subject.traits), "dialog_so_far": dialog_so_far}
    )
    meta_q = QuestionFreeText(
        question_name="next_question",
        question_text="""
        These are the biographic details of the interview subject: {{ scenario.subject }}
        This is your current dialog with the interview subject: {{ scenario.dialog_so_far }}
        What question you would ask the interview subject next?
        """,
    )
    question_text = (
        meta_q.by(model)
        .by(researcher)
        .by(scenario)
        .run()
        .select("next_question")
        .first()
    )
    return question_text


def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name="question",
        question_text=f"""
        This is your current dialog with the interview subject: {dialog_so_far}.
        You are now being asked:"""
        + question_text,
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first()
    return response


def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)

    print(" \nQuestion: \n\n" + question_text + "\n\nResponse: \n\n" + response)

    return {"question": question_text, "response": response}


def dialog_to_string(d):
    return "\n".join(
        [f"Question: {d['question']}\nResponse: {d['response']}" for d in d]
    )


def clean_dict(d):
    """Convert dictionary to string and remove braces."""
    return str(d).replace("{", "").replace("}", "")


def summarize_interview(
    interview_subject_name,
    interview_subject_traits,
    interview_topic,
    dialog_so_far,
    researcher,
):
    summary_q = QuestionFreeText(
        question_name="summary",
        question_text=(
            f"You have just conducted the following interview of {interview_subject_name} "
            f"who has these traits: {clean_dict(interview_subject_traits)} "
            f"The topic of the interview was {interview_topic}. "
            f"Please draft a summary of the interview: {clean_dict(dialog_so_far)}"
        ),
    )
    themes_q = QuestionFreeText(
        question_name="themes", question_text="List the major themes of the interview."
    )
    survey = Survey([summary_q, themes_q]).set_full_memory_mode()
    results = survey.by(model).by(researcher).run()
    summary = results.select("summary").first()
    themes = results.select("themes").first()
    print("\n\nSummary:\n\n" + summary + "\n\nThemes:\n\n" + themes)


def conduct_interview(
    interview_subject_name, interview_subject_traits, interview_topic
):
    subject = construct_subject(
        name=interview_subject_name, traits=interview_subject_traits
    )
    researcher = construct_researcher(interview_topic=interview_topic)

    print(
        "\n\nInterview subject: "
        + interview_subject_name
        + "\n\nInterview topic: "
        + interview_topic
    )

    dialog_so_far = []

    for i in range(total_questions):
        result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
        dialog_so_far.append(result)

    summarize_interview(
        interview_subject_name,
        interview_subject_traits,
        interview_topic,
        dialog_so_far,
        researcher,
    )

## Conduct the interview

In [6]:
conduct_interview(interview_subject_name, interview_subject_traits, interview_topic)

Interview subject: Chicken

Interview topic: Reasons to cross the road

Job UUID,ad5851e8-db6f-4e40-973c-b7cccdd3ce7b
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/ad5851e8-db6f-4e40-973c-b7cccdd3ce7b
Exceptions Report URL,None
Results UUID,58a234b0-894b-40cd-b2da-ea2e51b35b61
Results URL,https://www.expectedparrot.com/content/58a234b0-894b-40cd-b2da-ea2e51b35b61


Job UUID,648c87ab-9351-4ca6-bd5c-68d3c2093cfe
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/648c87ab-9351-4ca6-bd5c-68d3c2093cfe
Exceptions Report URL,None
Results UUID,f94a3697-fd7b-4bc5-8dff-d5ce8c23a952
Results URL,https://www.expectedparrot.com/content/f94a3697-fd7b-4bc5-8dff-d5ce8c23a952


Question: 

Okay, given the interviewee's profile (a brave, independent-minded chicken with 12 chicks on a free-range farm), I 
wouldn't ask about complex economic or political reasons for crossing the road.  Instead, I'd start with something 
concrete and relatable to their experience:

"I understand you live on a free-range farm.  Can you tell me about a time you had to cross the road? What made you
decide to do that?"


This opens the door for several follow-up questions, depending on their answer.  For example:

* **If they mention finding food:** "What kind of food were you looking for?  Was it worth the risk of crossing the
road to find it?" (This probes their decision-making process weighing risk vs. reward).
* **If they mention avoiding a predator:** "What kind of predator were you avoiding?  How did you know it was safe 
to cross the road at that moment?" (This explores their awareness of danger and survival strategies).
* **If they mention their chicks:** "Did you have your chicks with you? How did you keep them safe while crossing?"
(This focuses on their maternal instincts and protective behaviors).
* **If they mention something unexpected:** "That's interesting! Can you tell me more about that?" (This encourages
them to elaborate and provides a natural flow to the conversation).


The key is to listen carefully to their initial response and tailor subsequent questions to delve deeper into their
motivations and experiences.  The goal is to understand their perspective on crossing the road from a chicken's 
point of view, not to impose human reasoning on their actions.

Response: 

(Clucks thoughtfully, pecking at the ground for a moment before answering)

Well,  let me tell you, crossing the road isn't something I take lightly. It's a serious undertaking, especially 
with twelve little chicks in tow!  There was this one time…  a particularly juicy-looking patch of grubs had 
appeared on the other side.  I could practically smell them from here, a real feast for a mama hen like myself.  
But, of course, there's always the danger of… well, *everything*.  Cars, hawks, that grumpy old tomcat from Farmer 
McGregor's place… you name it.


(Pauses, glancing around cautiously)

So, I had to assess the situation.  The traffic was light, thankfully, and I spotted a good vantage point from a 
slightly raised area of grass. I waited for a lull, then,  *cluck-cluck-cluck*, I herded my chicks across.  It was 
a tense few seconds, let me tell you! But worth it for those grubs.  They were plump and juicy, and the chicks 
certainly appreciated the extra protein.

Job UUID,ee0ef3f4-c8e5-438e-b209-e5a84db0a025
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/ee0ef3f4-c8e5-438e-b209-e5a84db0a025
Exceptions Report URL,None
Results UUID,60a32c29-db60-45f7-9b3f-d135b033e470
Results URL,https://www.expectedparrot.com/content/60a32c29-db60-45f7-9b3f-d135b033e470


...

Job UUID,a2b9a914-056d-4283-b5d3-2f1e07af8be5
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/a2b9a914-056d-4283-b5d3-2f1e07af8be5
Exceptions Report URL,None
Results UUID,d3b7fc70-afc8-4de3-9bae-889964e6684c
Results URL,https://www.expectedparrot.com/content/d3b7fc70-afc8-4de3-9bae-889964e6684c


Job UUID,847e537e-2f9b-424e-a309-1b2639cfb760
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/847e537e-2f9b-424e-a309-1b2639cfb760
Exceptions Report URL,None
Results UUID,df7782ff-c58c-464f-b7e4-9e62d1a3ffec
Results URL,https://www.expectedparrot.com/content/df7782ff-c58c-464f-b7e4-9e62d1a3ffec


Question: 

Given the chicken's detailed and emotionally resonant response about the lasting impact of the near-miss with the 
truck, my next question would focus on her adaptation strategies and the potential for future similar situations.  
I want to understand her resilience and her ability to learn from experience.  Therefore, I would ask:

"You've described how your behavior has changed since the incident with the truck.  You're more cautious now, and 
your chicks are more responsive to your warnings.  But what if you *have* to cross the road again – perhaps to find
water, or if a predator forces you to move?  What strategies will you use to keep yourself and your chicks safe in 
the future?  Have you developed any new techniques or approaches to crossing the road, given your experience?"

This question probes several important aspects:

* **Future planning:** It explores the chicken's proactive approach to future risks, revealing her ability to learn
and adapt.
* **Problem-solving:** It assesses her capacity to develop new strategies and solutions to overcome challenges.
* **Resilience:** It indirectly assesses her resilience in the face of trauma and her ability to cope with future 
stressful situations.
* **Long-term adaptation:** It explores the lasting impact of the experience and her ability to integrate the 
lesson learned into her daily life.

This open-ended question allows the chicken to elaborate on her future plans and demonstrate her problem-solving 
skills, offering valuable insights into her adaptability and resilience.  It moves beyond the immediate trauma to 
focus on her ability to learn and grow from the experience.

Response: 

(I peck thoughtfully at the ground, my gaze fixed on a particularly interesting-looking pebble.  The memory of that
truck still makes my feathers ruffle, but I won't let fear rule my life.  I have twelve chicks depending on me, 
after all.)

"That's... that's a good question.  It makes me think.  I can't *avoid* crossing the road forever.  There's a 
lovely stream on the other side, and sometimes the best foraging spots are over there, too.  So, I've been thinking
about how to make it safer.

(I puff out my chest a little, a hint of pride in my voice.)

First, I'm teaching the chicks to recognize the sounds of approaching vehicles – cars, trucks, even tractors.  I've
been making those warning clucks whenever I hear one, so they're learning to associate the sound with danger.  It's
like a little training exercise, really.  They're getting better at it every day.

Second, I'm looking for safer crossing points.  There's a small dip in the hedge near the old oak tree – it offers 
a bit more cover from view.  If I can time my crossing with a break in the traffic, and use that hedge for cover, 
it might reduce the risk.

And finally...  (I pause, a little hesitant) ...I've been practicing a new technique.  It's a bit more...  
*strategic*.  Instead of just crossing straight across, I'm going to try crossing in short bursts, using the 
roadside bushes as cover between each dash.  It'll take longer, but it might be safer.  It's like a little game of 
chicken...  but *I'm* the chicken who's winning this time.

(I give a confident cluck, then add with a softer tone)  It's still scary, of course.  But I have to be brave for 
my chicks.  They need their mama to be strong, even when she's scared.  We'll manage. We always do."

Job UUID,0858d3da-8493-44cc-90a3-559fdf5a5897
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/0858d3da-8493-44cc-90a3-559fdf5a5897
Exceptions Report URL,None
Results UUID,780e499f-842b-49c7-ae7e-c4a89bf1e62a
Results URL,https://www.expectedparrot.com/content/780e499f-842b-49c7-ae7e-c4a89bf1e62a


Summary:

Interview Summary: Chicken's Reasons for Crossing the Road

This interview explored the motivations and experiences of Chicken, a wild, free-range hen with twelve chicks, 
regarding her decisions to cross a road.  The interview employed a qualitative approach, focusing on Chicken's 
firsthand accounts and perspectives rather than imposing human interpretations.

The initial question prompted Chicken to recount a specific instance where she crossed the road to access a 
plentiful patch of grubs. This decision, she explained, involved a careful risk assessment, weighing the potential 
dangers (cars, predators) against the reward (food for herself and her chicks).  She described her use of a vantage
point to observe traffic and her strategic herding of her chicks during the crossing.

Follow-up questions delved into her maternal instincts and protective behaviors. Chicken vividly recounted a 
near-miss with a truck, highlighting the intense stress and fear experienced during this event.  Her detailed 
description showcased her quick thinking and protective actions, utilizing sharp clucking sounds and her body to 
shield her chicks.  She described the immediate aftermath, emphasizing her role in calming and reassuring her 
frightened chicks.

The interview further explored the long-term impact of this traumatic event. Chicken acknowledged significant 
behavioral changes, including increased caution, heightened sensitivity to noise, and modified crossing strategies.
She described her chicks' increased wariness and their improved responsiveness to her warning calls.

Finally, the interview concluded by exploring Chicken's adaptive strategies for future crossings.  She outlined 
proactive measures, including teaching her chicks to recognize approaching vehicles, identifying safer crossing 
points, and developing a new technique involving short, covered dashes across the road.  Her responses demonstrated
resilience, problem-solving skills, and a strong maternal instinct driving her to adapt and protect her offspring.


The interview successfully captured Chicken's perspective, revealing her decision-making processes, protective 
behaviors, and adaptive capabilities in navigating the risks associated with crossing the road.  Her responses 
highlighted the interplay between risk assessment, maternal instincts, and the development of coping mechanisms in 
response to a traumatic experience.

Themes:

The major themes that emerged from the interview with Chicken are:

1. **Risk Assessment and Decision-Making:** Chicken's choices to cross the road involved a careful evaluation of 
potential dangers (predators, vehicles) versus the rewards (food, water, safer location).  This theme highlights 
her cognitive abilities and her capacity to weigh risks and benefits.

2. **Maternal Instincts and Protective Behaviors:** A significant portion of the interview focused on Chicken's 
protective behaviors towards her chicks.  Her actions during the near-miss with the truck vividly illustrated her 
strong maternal instincts and her problem-solving skills in ensuring their safety.

3. **Adaptation and Resilience:** The interview explored how Chicken adapted her behavior following a traumatic 
near-miss with a truck.  This theme highlights her capacity to learn from experience, modify her strategies, and 
develop new techniques to mitigate future risks.  Her resilience in the face of fear and her commitment to 
protecting her chicks were central to this theme.

4. **Communication and Training:** Chicken's ability to communicate with her chicks (using warning calls and body 
language) and to train them to recognize danger played a crucial role in their survival. This theme underscores the
importance of communication and learning within the flock.

5. **Emotional Response to Stress:** The interview revealed Chicken's emotional responses to stressful situations, 
such as fear and anxiety during the near-miss, and her subsequent coping mechanisms (